In [1]:
import os 
import pickle
from pathlib import Path
import yaml
from coxkan import CoxKAN
import sympy 
from sympy.printing.latex import latex

def format_cindex(input_string):
    return input_string[:9] + "\\newline " + input_string[9:]

def ex_round(ex1, floating_digit=4):
    ex2 = ex1
    for a in sympy.preorder_traversal(ex1):
        if isinstance(a, sympy.Float):
            ex2 = ex2.subs(a, round(a, floating_digit))
    return ex2

# Simulation studies (synthetic datasets)

In [2]:
import sympy
from sympy.printing.latex import latex

experiments = ['sim_gaussian','sim_depth_1', 'sim_deep', 'sim_difficult']

def format_cindex(input_string):
    return input_string[:9] + "\\newline " + input_string[9:]

print('True CoxPH CoxKAN_Symbolic')
for exp_name in experiments:
    with open(f'checkpoints/{exp_name}/results.pkl', 'rb') as f:
        results = pickle.load(f)

    formula = ex_round(results['coxkan_formula'], floating_digit=1)

    print('-------------------------')
    print(f'Experiment: {exp_name}')
    print(f'{format_cindex(results['cindex_true'])} & {format_cindex(results["cindex_cph"])} & {format_cindex(results["cindex_symbolic"])}')
    print(latex(formula))
    display(formula)
    print(results['coxph_formula'])

    if exp_name == 'sim_deep':
        formula = sympy.simplify(results['coxkan_formula'])
        formula = ex_round(formula, floating_digit=1)
        print(latex(formula))
        display(formula)


True CoxPH CoxKAN_Symbolic
-------------------------
Experiment: sim_gaussian
0.759744 \newline (0.759, 0.760) & 0.499213 \newline (0.497, 0.500) & 0.759739 \newline (0.760, 0.762)
5.0 e^{- 2.0 x_{1}^{2} - 2.0 x_{2}^{2}}


5.0*exp(-2.0*x1**2 - 2.0*x2**2)

-0.0009 * x1 + -0.0037 * x2 + 0.0159 * noise1 + -0.0134 * noise2
-------------------------
Experiment: sim_depth_1
0.759795 \newline (0.759, 0.761) & 0.688116 \newline (0.688, 0.690) & 0.759562 \newline (0.759, 0.761)
1.0 x_{3}^{2} - 1.0 \sin{\left(6.3 x_{2} - 9.4 \right)} + 1.0 \tanh{\left(5.1 x_{1} \right)}


1.0*x3**2 - 1.0*sin(6.3*x2 - 9.4) + 1.0*tanh(5.1*x1)

1.1208 * x1 + -0.3440 * x2 + 0.0096 * x3 + -0.0136 * noise1 + 0.0324 * noise2
-------------------------
Experiment: sim_deep
0.725470 \newline (0.725, 0.727) & 0.511198 \newline (0.508, 0.511) & 0.722706 \newline (0.721, 0.723)
4.0 \sqrt{x_{3}^{2} + 0.8 x_{4}^{2} + 0.9 \left(0.1 - x_{1}\right)^{2} + 1.0 \left(0.1 - x_{2}\right)^{2} - 0.5 \left(- 1.0 x_{1} - x_{2} + 0.1\right)^{2} - 0.7 \left(- x_{3} - 0.7 x_{4} - 0.1\right)^{2} + 0.6}


...

-0.0275 * x1 + 0.0507 * x2 + 0.0266 * x3 + 0.0120 * x4 + -0.0303 * noise1 + 0.0274 * noise2
3.9 \sqrt{0.5 x_{1}^{2} - x_{1} x_{2} + 0.5 x_{2}^{2} + 0.3 x_{3}^{2} - 1.0 x_{3} x_{4} + 0.5 x_{4}^{2} + 0.6}


3.9*sqrt(0.5*x1**2 - x1*x2 + 0.5*x2**2 + 0.3*x3**2 - 1.0*x3*x4 + 0.5*x4**2 + 0.6)

-------------------------
Experiment: sim_difficult
0.690174 \newline (0.689, 0.691) & 0.663698 \newline (0.663, 0.665) & 0.690127 \newline (0.689, 0.691)
- 1.0 \tanh{\left(-9.8 + 3.0 e^{- 2.6 x_{2}^{2}} + \frac{8.3}{\left(- 0.4 x_{1} - 1\right)^{3}} \right)}


-1.0*tanh(-9.8 + 3.0*exp(-2.6*x2**2) + 8.3/(-0.4*x1 - 1)**3)

2.0720 * x1 + 0.0511 * x2 + 0.0289 * noise1 + -0.0110 * noise2


# Real Clinical Data

In [3]:
import os 
import pickle
from pathlib import Path
import yaml
from coxkan import CoxKAN
import sympy

# datasets = ['gbsg', 'metabric', 'flchain', 'nwtco']
datasets = ['gbsg', 'metabric', 'support', 'flchain', 'nwtco']

def ex_round(ex1, floating_digit=4):
    ex2 = ex1
    for a in sympy.preorder_traversal(ex1):
        if isinstance(a, sympy.Float):
            ex2 = ex2.subs(a, round(a, floating_digit))
    return ex2

# Official results from DeepSurv publication (MLP-based survival model)
deepsurv_results = {
    'support': '0.618308 (0.616, 0.620)',
    'metabric': '0.643375 (0.639, 0.647)',
    'gbsg': '0.668402 (0.665, 0.671)',
}

incl_pruned = True

if incl_pruned:
    print('CoxPH DeepSurv CoxKAN_pre CoxKAN_pruned CoxKAN_Symbolic')
else:
    print('CoxPH DeepSurv CoxKAN_pre CoxKAN_Symbolic')
for dataset in datasets:

    directory = Path('checkpoints') / dataset
    with open(directory / 'results.pkl', 'rb') as f:
        results = pickle.load(f)

    with open(directory / 'config.yml', 'r') as f:
        config = yaml.safe_load(f)

    ckan = CoxKAN(seed=42, **config['init_params'])
    ckan.load_ckpt(directory / 'model.pt')
    normalizer = ckan.normalizer
    means = normalizer[0]
    stds = normalizer[1]

    ### C-Index results
    cindex_cph = results['CoxPH']['test']
    if 'DeepSurv' in results:
        cindex_deepsurv = results['DeepSurv']['test']
    else:
        cindex_deepsurv = deepsurv_results[dataset]
    cindex_pre = results['CoxKAN']['Pre']['test']
    cindex_pruned = results['CoxKAN']['Pruned']['test']
    cindex_symbolic = results['CoxKAN']['Symbolic']['test']

    print('\n-------------------------')
    print(f'Dataset: {dataset}')

    print(f'')

    if incl_pruned:
        print(f'{format_cindex(cindex_cph)} & {format_cindex(cindex_deepsurv)} & {format_cindex(cindex_pre)} & {format_cindex(cindex_pruned)} & {format_cindex(cindex_symbolic)}')
    else:
        print(f'{format_cindex(cindex_cph)} & {format_cindex(cindex_deepsurv)} & {format_cindex(cindex_pre)} & {format_cindex(cindex_symbolic)}')

    ### Formulas
    print('CoxKAN:')
    coxkan_formula = results['CoxKAN']['Symbolic']['formula']
    # coxkan_formula = ex_round(coxkan_formula, floating_digit=4)
    print('')
    if type(coxkan_formula) == str:
        print(coxkan_formula)
        print(latex(coxkan_formula))
    else:
        print(latex(coxkan_formula))
        display(coxkan_formula)

    print('CoxPH:')
    cph_coeffs = results['CoxPH']['summary']['coef']
    cph_formula = []
    for x, coef in cph_coeffs.items():
        mean = means[x]
        std = stds[x]
        x = (sympy.Symbol(x) - mean) / std
        cph_formula.append(ex_round(coef * x))
    cph_formula = sympy.Add(*cph_formula)
    display(cph_formula)
    print(latex(cph_formula))


CoxPH DeepSurv CoxKAN_pre CoxKAN_pruned CoxKAN_Symbolic
Loaded model from checkpoints/gbsg/model.pt

-------------------------
Dataset: gbsg

0.656291 \newline (0.655, 0.662) & 0.668402 \newline (0.665, 0.671) & 0.678294 \newline (0.676, 0.682) & 0.679219 \newline (0.675, 0.681) & 0.682796 \newline (0.678, 0.684)
CoxKAN:

- 0.0007 er + 1.8075 \left(1 - 0.0173 age\right)^{2} + 0.3979 \left(\begin{cases} -0.2933 & \text{for}\: meno = 0 \\0.5828 & \text{for}\: meno = 1.0 \\\text{NaN} & \text{otherwise} \end{cases}\right) + 0.3979 \left(\begin{cases} 0.6932 & \text{for}\: hormon = 0 \\-0.5338 & \text{for}\: hormon = 1.0 \\\text{NaN} & \text{otherwise} \end{cases}\right) + 0.3979 \left(\begin{cases} -0.1789 & \text{for}\: size = 0 \\0.5224 & \text{for}\: size = 1.0 \\1.2102 & \text{for}\: size = 2.0 \\\text{NaN} & \text{otherwise} \end{cases}\right) + 0.1049 \cosh{\left(0.0017 pgr - 1.6311 \right)} + 1.5829 - 1.2395 e^{- 0.0212 \left(- nodes - 0.3544\right)^{2}}


-0.0007*er + 1.8075*(1 - 0.0173*age)**2 + 0.3979*Piecewise((-0.2933, Eq(meno, 0)), (0.5828, Eq(meno, 1.0)), (nan, True)) + 0.3979*Piecewise((0.6932, Eq(hormon, 0)), (-0.5338, Eq(hormon, 1.0)), (nan, True)) + 0.3979*Piecewise((-0.1789, Eq(size, 0)), (0.5224, Eq(size, 1.0)), (1.2102, Eq(size, 2.0)), (nan, True)) + 0.1049*cosh(0.0017*pgr - 1.6311) + 1.5829 - 1.2395*exp(-0.0212*(-nodes - 0.3544)**2)

CoxPH:


0.0026*age - 0.0003*er - 0.3172*hormon + 0.2559*meno + 0.0557*nodes - 0.0003*pgr + 0.3379*size - 0.702

0.0026 age - 0.0003 er - 0.3172 hormon + 0.2559 meno + 0.0557 nodes - 0.0003 pgr + 0.3379 size - 0.702
Loaded model from checkpoints/metabric/model.pt

-------------------------
Dataset: metabric

0.632363 \newline (0.628, 0.637) & 0.643375 \newline (0.639, 0.647) & 0.647177 \newline (0.644, 0.652) & 0.648004 \newline (0.646, 0.654) & 0.649618 \newline (0.644, 0.651)
CoxKAN:

- 0.236 PGR + \begin{cases} -0.0524 & \text{for}\: chemo = 0 \\0.5889 & \text{for}\: chemo = 1.0 \\\text{NaN} & \text{otherwise} \end{cases} + \begin{cases} -0.0385 & \text{for}\: ER = 0 \\0.0722 & \text{for}\: ER = 1.0 \\\text{NaN} & \text{otherwise} \end{cases} + \begin{cases} 0.0294 & \text{for}\: hormone = 0 \\0.0942 & \text{for}\: hormone = 1.0 \\\text{NaN} & \text{otherwise} \end{cases} + \begin{cases} 0.1757 & \text{for}\: radio = 0 \\0.014 & \text{for}\: radio = 1.0 \\\text{NaN} & \text{otherwise} \end{cases} + 1.696 \sin{\left(0.0403 age - 9.5224 \right)} + 0.2347 \tanh{\left(1.86 MKI_{67} - 10.4299 \righ

...

0.1013 age + 0.0329 creatinine + 0.0551 flc.grp + 0.0154 \kappa + 0.1678 \lambda + 0.314 mgus + 0.0602 sample.yr + 0.2812 sex - 127.5629
Loaded model from checkpoints/nwtco/model.pt

-------------------------
Dataset: nwtco

0.698347 \newline (0.693, 0.703) & 0.698300 \newline (0.692, 0.703) & 0.719829 \newline (0.714, 0.725) & 0.720708 \newline (0.708, 0.718) & 0.722225 \newline (0.715, 0.725)
CoxKAN:

0.0167 age + 0.3472 \left(\begin{cases} -0.4069 & \text{for}\: 1 = histol \\1.6781 & \text{for}\: 2 = histol \\\text{NaN} & \text{otherwise} \end{cases}\right) + 0.1862 \left(\begin{cases} -0.3723 & \text{for}\: False = in.subcohort \\1.0526 & \text{for}\: True = in.subcohort \\\text{NaN} & \text{otherwise} \end{cases}\right) + 0.3472 \left(\begin{cases} -0.3668 & \text{for}\: 3 = study \\0.3606 & \text{for}\: 4 = study \\\text{NaN} & \text{otherwise} \end{cases}\right) + 0.1862 \left(\begin{cases} -0.3098 & \text{for}\: 1 = histol \\1.3512 & \text{for}\: 2 = histol \\\text{NaN} & \text

0.0167*age + 0.3472*Piecewise((-0.4069, Eq(1, histol)), (1.6781, Eq(2, histol)), (nan, True)) + 0.1862*Piecewise((-0.3723, Eq(False, in.subcohort)), (1.0526, Eq(True, in.subcohort)), (nan, True)) + 0.3472*Piecewise((-0.3668, Eq(3, study)), (0.3606, Eq(4, study)), (nan, True)) + 0.1862*Piecewise((-0.3098, Eq(1, histol)), (1.3512, Eq(2, histol)), (nan, True)) + 0.3472*Piecewise((-0.1453, Eq(1, instit)), (0.5484, Eq(2, instit)), (nan, True)) + 0.3895*Piecewise((-0.0544, Eq(1, histol)), (-0.5385, Eq(2, histol)), (nan, True)) + 0.3895*Piecewise((-0.0249, Eq(False, in.subcohort)), (0.4873, Eq(True, in.subcohort)), (nan, True)) + 0.3895*Piecewise((-0.001, Eq(3, study)), (0.0854, Eq(4, study)), (nan, True)) + 0.3895*Piecewise((0.0281, Eq(1, instit)), (-0.8385, Eq(2, instit)), (nan, True)) + 0.1862*Piecewise((0.1299, Eq(3, study)), (-0.1148, Eq(4, study)), (nan, True)) + 0.3472*Piecewise((-0.7809, Eq(1, stage)), (0.0716, Eq(2, stage)), (0.4768, Eq(3, stage)), (1.0507, Eq(4, stage)), (nan, True)

CoxPH:


0.0064*age + 1.4956*histol - 0.2192*in.subcohort + 0.2163*instit + 0.3413*stage - 0.1407*study - 0.7196

0.0064 age + 1.4956 histol - 0.2192 in.subcohort + 0.2163 instit + 0.3413 stage - 0.1407 study - 0.7196


#### METABRIC

$$
\begin{aligned}
\hat{\theta}_{KAN} = 
        & - 0.24 \cdot \text{PGR}
        + 0.7 e^{- 26 \left(1 - 0.06 \cdot \text{ERBB2}\right)^{2}} \\
        & + 0.2 \tanh(1.9 \text{MKI67} - 10)
        +\left\{ 
        \begin{array}{ll}
        0.1 & \text{if } \text{hormonal therapy} \\
        0.03 & \text{otherwise}
        \end{array}
        \right\} \\
        & +\left\{ 
        \begin{array}{ll}
        0.01 & \text{if } \text{radiotherapy} \\
        0.18 & \text{otherwise}
        \end{array}
        \right\}
        +\left\{ 
        \begin{array}{ll}
        0.6 & \text{if } \text{chemotherapy} \\
        -0.05 & \text{otherwise}
        \end{array}
        \right\} \\
        & +\left\{ 
        \begin{array}{ll}
        0.07 & \text{if } \text{ER positive} \\
        -0.04 & \text{otherwise}
        \end{array}
        \right\}
        - 1.7 \sin{\left(0.04 \cdot \text{Age} - 9.5 \right)} \\
\end{aligned}
$$

$$
\hat{\theta}_{CPH} = 0.0422 \text{EGFR} + 0.0693 \text{ER} + 0.1023 \text{ERBB2} + 0.3145 \text{MKI67} - 0.0698 \text{PGR} + 0.0433 \text{age} + 0.7712 \text{chemo} + 0.1849 \text{hormone} - 0.2118 \text{radio} 
$$

### SUPPORT

$$
\begin{aligned}
\hat{\theta}_{KAN} = 

        & \, \phi_{interact} - 0.0002 \cdot \text{age}
        +\left\{ 
        \begin{array}{ll}
        0.003 & \text{if } \text{metastasis} \\
        -0.01 & \text{if } \text{no cancer} \\
        -0.0098 & \text{if } \text{cancer}
        \end{array}
        \right\} \\
        & +\left\{ 
        \begin{array}{ll}
        0.007 & \text{if } \text{male} \\
        -0.01 & \text{if } \text{female}
        \end{array}
        \right\}
        - 0.01 \cdot \text{race} + 0.04 \cdot \text{comorbidity} \\
        & +\left\{ 
        \begin{array}{ll}
        -0.03 & \text{if } \text{diabetes} \\
        0.0006 & \text{if } \text{diabetes}
        \end{array}
        \right\} \\
        & +\left\{ 
        \begin{array}{ll}
        0.03 & \text{if } \text{dementia} \\
        -0.0008 & \text{if } \text{dementia}
        \end{array}
        \right\} \\
        & + 0.9 e^{- 0.06 \left(1 - 0.1 \cdot \text{meanbp}\right)^{2}}
        + 0.1 \tanh{\left(0.02 \cdot \text{hr} - 3 \right)} \\
        & - 0.06 \sin{\left(0.08 \cdot \text{rr} + 0.2 \right)}
        + 0.6 e^{- 572 \left(1 - 0.02 \cdot \text{temp}\right)^{2}} \\
        & + 0.0008 \cdot \text{sodium}
        + 0.03 \tan{\left(0.02 \cdot \text{wbc} - 4 \right)}
        + 0.003 \cdot \text{creatinine} \\
\end{aligned}
$$

$$
\begin{aligned}
\hat{\theta}_{CPH} = & \, 0.01 \cdot \text{age} - 0.09 \cdot \text{sex} + 0.02 \cdot \text{race} + 0.02 \cdot \text{comorbidity} - 0.05 \cdot \text{diabetes} \\ 
& + 0.1 \cdot \text{dementia} - 0.26 \cdot \text{cancer} - 0.003 \cdot \text{meanbp} + 0.002 \cdot \text{hr} \\
& + 0.002 \cdot \text{rr} + 0.01 \cdot \text{temp} - 0.004 \cdot \text{sodium} + 0.003 \cdot \text{wbc} + 0.03 \cdot \text{creatinine}
\end{aligned}
$$

### NWTCO

$$
\begin{aligned}
\hat{\theta}_{KAN} = 
        & \, \phi_{1,3,1} + \phi_{1,4,1} + 0.02 \cdot \text{age}  \\
        & +\left\{ 
        \begin{array}{ll}
        -0.047 & \text{if } \text{favourable histology (instit)} \\
        -0.136 & \text{if } \text{unfavourable histology (instit)}
        \end{array}
        \right\} \\
        & +\left\{ 
        \begin{array}{ll}
        -0.22 & \text{if } \text{favourable histology (histol)} \\
        0.62 & \text{if } \text{unfavourable histology (histol)}
        \end{array}
        \right\} \\
        & +\left\{ 
        \begin{array}{ll}
        -0.47 & \text{if } \text{stage } = 1 \\
         0.04 & \text{if } \text{stage } = 2 \\
         0.35 & \text{if } \text{stage } = 3 \\
         0.78 & \text{if } \text{stage } = 4 \\
        \end{array}
        \right\} \\
        & +\left\{ 
        \begin{array}{ll}
        0.02 & \text{if } 3^{rd} \text{ clinical study} \\
        0.01 & \text{if } 4^{th} \text{ clinical study}
        \end{array}
        \right\} \\
        & + \left\{
        \begin{array}{ll}
        0.2 & \text{if } \text{in subcohort} \\
        -0.07 & \text{otherwise}
        \end{array}
        \right\}
    
\end{aligned}
$$

$$
\begin{aligned}
\phi_{1,3,1} = - 2.5 \arctan \Bigg( 2 \Bigg[
        & +0.03 \cdot \text{age} \\
        & +\left\{ 
        \begin{array}{ll}
        -0.1 & \text{if } \text{favourable histology (instit)} \\
        -0.4 & \text{if } \text{unfavourable histology (instit)}
        \end{array}
        \right\} \\
        & +\left\{ 
        \begin{array}{ll}
        0.3 & \text{if } \text{favourable histology (histol)} \\
        -0.4 & \text{if } \text{unfavourable histology (histol)}
        \end{array}
        \right\} \\
        & +\left\{ 
        \begin{array}{ll}
        0.1 & \text{if } \text{stage } = 1 \\
        -0.07 & \text{if } \text{stage } = 2 \\
        -0.03 & \text{if } \text{stage } = 3 \\
        -0.17 & \text{if } \text{stage } = 4 \\
        \end{array}
        \right\} \\
        & +\left\{ 
        \begin{array}{ll}
        -0.2 & \text{if } 3^{rd} \text{ clinical study} \\
        0.09 & \text{if } 4^{th} \text{ clinical study}
        \end{array}
        \right\} \\

        & + \left\{
        \begin{array}{ll}
        1 & \text{if } \text{in subcohort} \\
        -0.3 & \text{otherwise}
        \end{array}
        \right\}
\Bigg] \Bigg)
\end{aligned}
$$

$$
\begin{aligned}
\phi_{1,4,1} = - 6 \tanh \Bigg( 0.7 \Bigg[
        & +0.006 \cdot \text{age} \\
        & +\left\{ 
        \begin{array}{ll}
        0.15 & \text{if } \text{favourable histology (instit)} \\
        -0.7 & \text{if } \text{unfavourable histology (instit)}
        \end{array}
        \right\} \\
        & +\left\{ 
        \begin{array}{ll}
        0.1 & \text{if } \text{favourable histology (histol)} \\
        0.01 & \text{if } \text{unfavourable histology (histol)}
        \end{array}
        \right\} \\
        & +\left\{ 
        \begin{array}{ll}
        -0.7 & \text{if } \text{stage } = 1 \\
        -0.2 & \text{if } \text{stage } = 2 \\
        0.7 & \text{if } \text{stage } = 3 \\
        1.45 & \text{if } \text{stage } = 4 \\
        \end{array}
        \right\} \\
        & +\left\{ 
        \begin{array}{ll}
        -0.6 & \text{if } 3^{rd} \text{ clinical study} \\
        0.5 & \text{if } 4^{th} \text{ clinical study}
        \end{array}
        \right\} \\

        & + \left\{
        \begin{array}{ll}
        1 & \text{if } \text{in subcohort} \\
        -0.2 & \text{otherwise}
        \end{array}
        \right\}
\Bigg] \Bigg)
\end{aligned}
$$

$$
\begin{aligned}
\hat{\theta}_{CPH} = & \, + 0.2163 \cdot \text{instit} + 1.4956 \cdot \text{histol} + 0.3413 \cdot \text{stage} \\
& - 0.1407 \cdot \text{study} + 0.0064 \cdot \text{age} - 0.2192 \cdot \text{in.subcohort}
\end{aligned}
$$

#### FLCHAIN

$$
\begin{aligned}
\hat{\theta}_{KAN} =  & \, 0.09 \cdot \text{age}         
        +\left\{ 
        \begin{array}{ll}
        -0.047 & \text{if } \text{female} \\
        0.118 & \text{if } \text{male}
        \end{array}
        \right\} \\
        & + 0.4 \arctan(0.4 \cdot \text{year} - 737)
        + 0.04 \cdot \text{FLC}_{kappa} \\
        & + 0.3 \cdot \text{FLC}_{lambda} 
        + 0.009 \cdot \text{FLC}_\text{group}
        + 2\arctan(0.5 \cdot \text{creatinine} - 0.9) \\
\end{aligned}
$$

$$
\hat{\theta}_{CPH} = 0.1 \cdot \text{age} + 0.3 \cdot \text{sex} + 0.06 \cdot \text{year} + 0.01 \cdot \text{FLC}_{kappa} + 0.2 \cdot \text{FLC}_{lambda} + 0.06 \cdot \text{FLC}_\text{group} + 0.03 \cdot \text{creatinine} + 0.3 \cdot \text{mgus}
$$

#### GBSG

$$
\begin{aligned}
\hat{\theta}_{KAN} = 
    & + \left\{ 
        \begin{array}{ll}
        -0.21 & \text{if } \text{hormonal therapy} \\
        0.28 & \text{otherwise}
        \end{array}
        \right\} \\
    & +\left\{
        \begin{array}{ll}
        -0.07 & \text{if } \text{tumor size} \leq 20 \, \text{mm} \\
        0.21 & \text{if } 20 < \text{tumor size} < 50 \, \text{mm} \\
        0.48 & \text{if } \text{tumor size} \geq 50 \, \text{mm}
        \end{array}
        \right\} \\
    & +\left\{ 
        \begin{array}{ll}
        -0.12 & \text{if } \text{pre-menopausal} \\
        0.23 & \text{if } \text{post-menopausal}
        \end{array}
        \right\} \\
    & + 1.8 \left(1 - 0.02 \cdot \text{age}\right)^{2} \\ 
    & - 1.2 e^{- 0.02 \left(\text{nodes} + 0.4\right)^{2}} \\
    & + 0.1 \cosh{\left(0.002 \cdot \text{PGR} - 1.6 \right)} \\
    & - 0.0007 \cdot \text{ER}
\end{aligned}
$$

$$
\hat{\theta}_{CPH} = 0.003 \cdot \text{age} - 0.0003 \cdot \text{er} - 0.3 \cdot \text{hormon} + 0.26 \cdot \text{meno} + 0.06 \cdot \text{nodes} - 0.0003 \cdot \text{pgr} + 0.3 \cdot \text{size}
$$


# TCGA Genomics

In [5]:
import pickle
from sympy.printing.latex import latex

experiments = ['TCGA-STAD', 'TCGA-BRCA', 'TCGA-GBMLGG', 'TCGA-KIRC']

print('CoxPH  CoxPH Reg  DeepSurv  CoxKAN Pre  CoxKAN Pruned  CoxKAN Symbolic')
for exp_name in experiments:

    results = pickle.load(open(f'checkpoints/{exp_name}/results.pkl', 'rb'))

    print('-------------------------')
    print(f'Experiment: {exp_name}')

    try: cindex_cph = results['CoxPH']['test']
    except: cindex_cph = 'N/A '
    cindex_cph_reg = results['CoxPH Reg']['test']
    cindex_deepsurv = results['DeepSurv']['test']
    cindex_pre = results['CoxKAN']['Pre']['test']
    cindex_pruned = results['CoxKAN']['Pruned']['test']
    cindex_symbolic = results['CoxKAN']['Symbolic']['test']

    print(f'{exp_name[5:]} & {format_cindex(cindex_cph)} & {format_cindex(cindex_cph_reg)} & {format_cindex(cindex_deepsurv)} & {format_cindex(cindex_pre)} & {format_cindex(cindex_pruned)} & {format_cindex(cindex_symbolic)}')


CoxPH  CoxPH Reg  DeepSurv  CoxKAN Pre  CoxKAN Pruned  CoxKAN Symbolic
-------------------------
Experiment: TCGA-STAD
STAD & 0.543441 \newline (0.521, 0.543) & 0.677172 \newline (0.673, 0.694) & 0.628620 \newline (0.616, 0.638) & 0.700170 \newline (0.697, 0.715) & 0.670358 \newline (0.665, 0.690) & 0.671210 \newline (0.659, 0.682)
-------------------------
Experiment: TCGA-BRCA
BRCA & 0.539545 \newline (0.529, 0.560) & 0.613182 \newline (0.607, 0.635) & 0.632500 \newline (0.623, 0.648) & 0.619091 \newline (0.593, 0.621) & 0.634545 \newline (0.617, 0.638) & 0.630455 \newline (0.622, 0.642)
-------------------------
Experiment: TCGA-GBMLGG
GBMLGG & N/A \newline  & 0.787844 \newline (0.777, 0.799) & 0.819094 \newline (0.820, 0.836) & 0.813647 \newline (0.808, 0.824) & 0.811353 \newline (0.804, 0.820) & 0.818234 \newline (0.817, 0.828)
-------------------------
Experiment: TCGA-KIRC
KIRC & N/A \newline  & 0.686285 \newline (0.663, 0.686) & 0.650378 \newline (0.641, 0.664) & 0.672246 \newl

In [6]:
exp_name = 'TCGA-STAD'
results = pickle.load(open(f'checkpoints/{exp_name}/results.pkl', 'rb'))

coxkan_formula = results['CoxKAN']['Symbolic']['formula']
display(coxkan_formula)

print(latex(coxkan_formula))

{k:v for k,v in list(results['CoxKAN']['Symbolic']['terms_std'].items())[:10]}

-0.0896*ANK3_(ENSG00000151150)_mut + 0.1657*APOA1_(ENSG00000118137.10)_rna + 0.043*APOD_(ENSG00000189058.9)_rna - 0.048*ATM_(ENSG00000149311)_mut - 0.1149*CDH23_(ENSG00000107736)_mut + 0.101*CKS2_(ENSG00000123975.5)_rna + 0.0302*COL11A1_(ENSG00000060718)_mut - 0.0937*COL6A2_(ENSG00000142173.16)_rna + 0.045*DCLK1_(ENSG00000133083)_mut - 0.0658*DIDO1_(ENSG00000101191)_mut + 0.0746*DST_(ENSG00000151914)_mut - 0.1006*FAT2_(ENSG00000086570)_mut - 0.0429*FLG_(ENSG00000143631)_mut + 0.0057*GLI3_(ENSG00000106571)_mut - 0.1104*HNRNPK_(ENSG00000165119.21)_rna - 0.0241*MACF1_(ENSG00000127603)_mut - 0.1246*MISP_(ENSG00000099812.9)_rna - 0.0886*MUC16_(ENSG00000181143)_mut + 0.1991*MUCL3_(ENSG00000168631.13)_rna - 0.0914*MYO16_(ENSG00000041515)_mut - 0.0554*NRXN1_(ENSG00000179915)_mut + 0.0803*POSTN_(ENSG00000133110.15)_rna - 0.1382*PRR15L_(ENSG00000167183.3)_rna - 0.0862*PTPRT_(ENSG00000196090)_mut + 0.069*SERINC1_(ENSG00000111897.7)_rna + 0.1379*SPINK1_(ENSG00000164266.11)_rna + 0.1105*STOM_(ENSG0

- 0.0896 ANK_{3 (ENSG00000151150) mut} + 0.1657 APOA_{1 (ENSG00000118137.10) rna} + 0.043 APOD_{(ENSG00000189058.9) rna} - 0.048 ATM_{(ENSG00000149311) mut} - 0.1149 CDH_{23 (ENSG00000107736) mut} + 0.101 CKS_{2 (ENSG00000123975.5) rna} + 0.0302 COL11A1_{(ENSG00000060718) mut} - 0.0937 COL6A2_{(ENSG00000142173.16) rna} + 0.045 DCLK_{1 (ENSG00000133083) mut} - 0.0658 DIDO_{1 (ENSG00000101191) mut} + 0.0746 DST_{(ENSG00000151914) mut} - 0.1006 FAT_{2 (ENSG00000086570) mut} - 0.0429 FLG_{(ENSG00000143631) mut} + 0.0057 GLI_{3 (ENSG00000106571) mut} - 0.1104 HNRNPK_{(ENSG00000165119.21) rna} - 0.0241 MACF_{1 (ENSG00000127603) mut} - 0.1246 MISP_{(ENSG00000099812.9) rna} - 0.0886 MUC_{16 (ENSG00000181143) mut} + 0.1991 MUCL_{3 (ENSG00000168631.13) rna} - 0.0914 MYO_{16 (ENSG00000041515) mut} - 0.0554 NRXN_{1 (ENSG00000179915) mut} + 0.0803 POSTN_{(ENSG00000133110.15) rna} - 0.1382 PRR15L_{(ENSG00000167183.3) rna} - 0.0862 PTPRT_{(ENSG00000196090) mut} + 0.069 SERINC_{1 (ENSG00000111897.7) r

{'(0,95,0) 0.24425*tanh(0.9588*CALM2_(ENSG00000143933.19)_rna - 0.406)': 0.14763517582456648,
 '(0,76,0) -0.13818*PRR15L_(ENSG00000167183.3)_rna - 0.13265': 0.10607886742645557,
 '(0,78,0) 0.23288*TOMM20_(ENSG00000173726.11)_rna + 0.20028': 0.08998552750768095,
 '(0,140,0) 0.08509 - 0.08863*MUC16_(ENSG00000181143)_mut': 0.0886307223101623,
 '(0,127,0) 0.75749*atan(0.366*C3_(ENSG00000125730.17)_rna + 0.17896)': 0.07924222220372616,
 '(0,99,0) -0.11039*HNRNPK_(ENSG00000165119.21)_rna - 0.09273': 0.07898811363438991,
 '(0,116,0) 0.10712 - 0.12456*MISP_(ENSG00000099812.9)_rna': 0.07522894236203936,
 '(0,123,0) 0.10103*CKS2_(ENSG00000123975.5)_rna + 0.09497': 0.06674894594438927,
 '(0,79,0) 0.11049*STOM_(ENSG00000148175.13)_rna + 0.10386': 0.06089391191990403,
 '(0,117,0) 0.17986*atan(0.53488*FAM83H_(ENSG00000180921.7)_rna + 0.66456)': 0.05722617064206513}

$$
\begin{split}
\hat{\theta}_{KAN} = & + 0.2 \tanh(\text{CALM2}_{RNA} - 0.4) \quad (\sigma=0.15) \, \\
& -0.1 \cdot \text{PRR15L}_{RNA} \quad (\sigma=0.1) \, \\
& + 0.2 \cdot \text{TOMM20}_{RNA} \quad (\sigma=0.09) \, \\
& - 0.09 \cdot \text{MUC16}_{mut} \quad (\sigma=0.09) \, \\
& + 0.8 \arctan(0.4 \cdot \text{C3}_{RNA} + 0.2) \quad (\sigma=0.08) \, \\
& -0.1 \cdot \text{HNRNPK}_{RNA} \quad (\sigma=0.08) \, \\
& -0.2 \cdot \text{MISP}_{RNA} \quad (\sigma=0.08) \, \\
& + \text{less significant terms},
\end{split}
$$

In [7]:
exp_name = 'TCGA-BRCA'
results = pickle.load(open(f'checkpoints/{exp_name}/results.pkl', 'rb'))

coxkan_formula = results['CoxKAN']['Symbolic']['formula']
display(coxkan_formula)

print(latex(coxkan_formula))

{k:v for k,v in list(results['CoxKAN']['Symbolic']['terms_std'].items())[:10]}

0.1054*DMD_(ENSG00000198947)_mut + 0.2485*KMT2C_(ENSG00000055609)_mut - 0.1401*RYR2_(ENSG00000198626)_mut + 0.0733*TTN_(ENSG00000155657)_mut + 0.0505*sin(1.1065*AP002791.1_(ENSG00000280093)_cnv - 5.0262) - 0.1775*sin(0.6052*CHMP4B_(ENSG00000101421.4)_rna + 5.0074) + 0.6474*sin(0.4853*HSPA8_(ENSG00000109971.14)_rna - 7.3478) - 0.293*sin(0.5062*RPL14_(ENSG00000188846.14)_rna + 4.9681) + 0.4716*sin(0.512*group_40_cnv + 5.1666) + 0.1539 - 2.2206*exp(-0.0419*(-0.8597*PLXNB2_(ENSG00000196576.15)_rna - 1)**2) - 2.0795*exp(-0.0474*(-0.8692*PGK1_(ENSG00000102144.15)_rna - 1)**2) + 0.9466*exp(-0.0633*(-H2BC5_(ENSG00000158373.8)_rna - 0.451)**2) + 0.386/(1 - 0.0952*group_46_cnv)**2

0.1054 DMD_{(ENSG00000198947) mut} + 0.2485 KMT2C_{(ENSG00000055609) mut} - 0.1401 RYR_{2 (ENSG00000198626) mut} + 0.0733 TTN_{(ENSG00000155657) mut} + 0.0505 \sin{\left(1.1065 AP002791.1_{(ENSG00000280093) cnv} - 5.0262 \right)} - 0.1775 \sin{\left(0.6052 CHMP4B_{(ENSG00000101421.4) rna} + 5.0074 \right)} + 0.6474 \sin{\left(0.4853 HSPA_{8 (ENSG00000109971.14) rna} - 7.3478 \right)} - 0.293 \sin{\left(0.5062 RPL_{14 (ENSG00000188846.14) rna} + 4.9681 \right)} + 0.4716 \sin{\left(0.512 group_{40 cnv} + 5.1666 \right)} + 0.1539 - 2.2206 e^{- 0.0419 \left(- 0.8597 PLXNB_{2 (ENSG00000196576.15) rna} - 1\right)^{2}} - 2.0795 e^{- 0.0474 \left(- 0.8692 PGK_{1 (ENSG00000102144.15) rna} - 1\right)^{2}} + 0.9466 e^{- 0.0633 \left(- H2BC5_{(ENSG00000158373.8) rna} - 0.451\right)^{2}} + \frac{0.386}{\left(1 - 0.0952 group_{46 cnv}\right)^{2}}


{'(0,164,0) 0.24847*KMT2C_(ENSG00000055609)_mut + 0.20871': 0.24218513703055525,
 '(0,99,0) 0.64738*sin(0.48528*HSPA8_(ENSG00000109971.14)_rna - 7.34776)': 0.1781848872763935,
 '(0,158,0) -2.22062*exp(-0.04191*(-0.85971*PLXNB2_(ENSG00000196576.15)_rna - 1)**2)': 0.16922728756722222,
 '(0,75,0) -2.07947*exp(-0.04742*(-0.86921*PGK1_(ENSG00000102144.15)_rna - 1)**2)': 0.15344060125528117,
 '(0,167,0) -0.14008*RYR2_(ENSG00000198626)_mut - 0.14008': 0.138683688702026,
 '(0,166,0) 0.10544*DMD_(ENSG00000198947)_mut + 0.09701': 0.10149801146122837,
 '(0,165,0) 0.07327*TTN_(ENSG00000155657)_mut - 0.01172': 0.0743229478074131,
 '(0,57,0) 0.38596/(1 - 0.09523*group_46_cnv)**2': 0.06272604925424385,
 '(0,110,0) 0.9466*exp(-0.06334*(-H2BC5_(ENSG00000158373.8)_rna - 0.45105)**2)': 0.05471130865454245,
 '(0,108,0) -0.29297*sin(0.50616*RPL14_(ENSG00000188846.14)_rna + 4.96808)': 0.04758794434583577}

$$
\begin{equation}
\begin{split}
\hat{\theta}_{KAN} = & + 0.2 \cdot \text{KMT2C}_{mut} \quad (\sigma=0.24) \, \\
& + 0.6 \sin(0.5 \cdot \text{HSPA8}_{RNA} - 7) \quad (\sigma=0.18)\, \\
& -2 \exp{(-0.04 (0.9 \cdot \text{PLXNB2}_{RNA} + 1)^2)} \quad (\sigma=0.17) \, \\
& -2 \exp{(-0.05 (0.9 \cdot \text{PGK1}_{RNA} + 1)^2)} \quad (\sigma=0.15) \, \\
& -0.14 \cdot \text{RYR2}_{mut} \quad (\sigma=0.14) \, \\
& +0.1 \cdot \text{DMD}_{mut} \quad (\sigma=0.1) \, \\
& +0.01 \text{TTN}_{mut} \quad (\sigma=0.07) \, \\
& + \frac{0.4}{(1-0.1 \cdot \text{group\_46}_{CNV})^2} \quad (\sigma=0.06) \, \\
& + 0.9 \exp(-0.06(\text{H2BC5}_{RNA} - 0.5)^2) \quad (\sigma=0.05) \, \\
& - 0.3 \sin(0.5 \cdot \text{RPL14}_{RNA} + 5) \quad (\sigma=0.05) \, \\
& + \text{less significant terms},
\end{split}
\end{equation}
$$

In [8]:
exp_name = 'TCGA-GBMLGG'
results = pickle.load(open(f'checkpoints/{exp_name}/results.pkl', 'rb'))

coxkan_formula = results['CoxKAN']['Symbolic']['formula']
display(coxkan_formula)

print(latex(coxkan_formula))

{k:v for k,v in list(results['CoxKAN']['Symbolic']['terms_std'].items())[:10]}

-0.1573*10p - 0.1481*CALN1_rnaseq - 0.1184*CDKN2A + 0.1646*DNAH17_rnaseq + 0.215*FRAS1_rnaseq - 0.1893*codeletion - 0.1682*idh_mutation - 0.0752*(-0.614*PTEN - 1)**4 - 0.1278*sin(8.9694*CDKN2B - 3.828) - 0.3167*sin(8.6039*EGFR + 0.8108) - 0.2616*sin(2.778*JAK2 - 4.5936) + 0.3336*sin(0.8299*LRP1B_rnaseq + 2.0076) + 6.4865*sin(1.0555*UGT2B4_rnaseq + 8.0027) - 0.062*tan(0.4191*CARD11 + 7.5848) + 0.4082*atan(0.5956*L1CAM_rnaseq + 0.1044) - 0.2674 + 0.9989*exp(-0.1283*(-RB1_rnaseq - 0.9004)**2) + 1.2852*exp(-0.2444*(-0.5966*10q - 1)**2) + 3861.3083*exp(-43.446*(1 - 0.0947*DNAH11_rnaseq)**2) - 1.8894/(3.2094 - 0.6035*MUC17_rnaseq) - 3.0631/(1 - 0.0755*TEKT4_rnaseq)**3

- 0.1573 \cdot 10p - 0.1481 CALN_{1 rnaseq} - 0.1184 CDKN2A + 0.1646 DNAH_{17 rnaseq} + 0.215 FRAS_{1 rnaseq} - 0.1893 codeletion - 0.1682 idh_{mutation} - 0.0752 \left(- 0.614 PTEN - 1\right)^{4} - 0.1278 \sin{\left(8.9694 CDKN2B - 3.828 \right)} - 0.3167 \sin{\left(8.6039 EGFR + 0.8108 \right)} - 0.2616 \sin{\left(2.778 JAK_{2} - 4.5936 \right)} + 0.3336 \sin{\left(0.8299 LRP1B_{rnaseq} + 2.0076 \right)} + 6.4865 \sin{\left(1.0555 UGT2B4_{rnaseq} + 8.0027 \right)} - 0.062 \tan{\left(0.4191 CARD_{11} + 7.5848 \right)} + 0.4082 \operatorname{atan}{\left(0.5956 L1CAM_{rnaseq} + 0.1044 \right)} - 0.2674 + 0.9989 e^{- 0.1283 \left(- RB_{1 rnaseq} - 0.9004\right)^{2}} + 1.2852 e^{- 0.2444 \left(- 0.5966 \cdot 10q - 1\right)^{2}} + 3861.3083 e^{- 43.446 \left(1 - 0.0947 DNAH_{11 rnaseq}\right)^{2}} - \frac{1.8894}{3.2094 - 0.6035 MUC_{17 rnaseq}} - \frac{3.0631}{\left(1 - 0.0755 TEKT_{4 rnaseq}\right)^{3}}


{'(0,0,0) -0.18926*codeletion - 0.14762': 0.19140902331173626,
 '(0,61,0) 1.28525*exp(-0.24443*(-0.5966*10q - 1)**2)': 0.19017926715286923,
 '(0,1,0) -0.16824*idh_mutation - 0.14805': 0.16770168876090089,
 '(0,39,0) -0.062*tan(0.41912*CARD11 + 7.5848)': 0.15811140049172392,
 '(0,42,0) -0.07521*(-0.61398*PTEN - 1)**4': 0.1392790235227957,
 '(0,40,0) -0.26156*sin(2.778*JAK2 - 4.5936)': 0.12395199984459931,
 '(0,16,0) 0.0663 - 0.11839*CDKN2A': 0.12181632968237227,
 '(0,313,0) -3.06314/(1 - 0.07545*TEKT4_rnaseq)**3': 0.09913908563391044,
 '(0,17,0) -0.12779*sin(8.96936*CDKN2B - 3.828)': 0.09636957815660824,
 '(0,2,0) -0.31668*sin(8.60392*EGFR + 0.8108)': 0.07970619573938781}


\begin{equation}
\begin{split}
\hat{\theta}_{KAN} = 
& \, - 0.2 \cdot \text{(1p19q arm codeletion)} \quad (\sigma=0.19) \, \\
& +e^{-0.2(-0.6 \cdot (\text{10q}_{CNV}) - 1)^2} \quad (\sigma=0.19) \, \\
& -0.2 \cdot \text{IDH}_{mut} \quad (\sigma=0.17) \, \\
& -0.06 \tan(0.4 \cdot \text{CARD11}_{CNV} + 8) \quad (\sigma=0.16) \, \\
& -0.08 (0.6 \cdot \text{PTEN}_{CNV} + 1)^4 \quad (\sigma=0.14) \, \\
& -0.3 \sin(3 \cdot \text{JAK2}_{CNV} - 5) \quad (\sigma=0.12) \, \\
& - 0.1 \cdot \text{CDKN2A}_{CNV} \quad (\sigma=0.12) \, \\
& -0.1 \sin(9 \cdot \text{CDKN2B}_{CNV} - 4) \quad (\sigma=0.1) \, \\
& - 0.3 \sin(9 \cdot \text{EGFR}_{CNV} + 0.8) \quad (\sigma=0.1) \, \\
& + \text{less significant terms},
\end{split}
\end{equation}

In [13]:
exp_name = 'TCGA-KIRC'
results = pickle.load(open(f'checkpoints/{exp_name}/results.pkl', 'rb'))

coxkan_formula = results['CoxKAN']['Symbolic']['formula']
display(coxkan_formula)

print(latex(coxkan_formula))

# {k:v for k,v in list(results['CoxKAN']['Symbolic']['terms_std'].items())[:10]}

'-0.0224*AACSP1_cnv + 0.0164*ACHE_rna - 0.1246*ACTG2_rna - 0.2214*ADAM6_rna + 0.0751*ADAMTS2_cnv + 0.0429*ADCYAP1_rna + 0.092*ADRB1_rna - 0.0548*AGR2_rna - 0.0251*AGT_rna - 0.0959*AGXT2_rna - 0.1538*ALPI_rna - 0.0033*ANKRD1_rna + 0.3331*APCDD1L_rna - 0.2505*AREG_rna - 0.2206*ARL10_cnv + 0.1533*ARPP21_rna - 0.2197*ART5_rna - 0.0648*ASAM_rna + 0.024*ATP13A4_rna - 0.1172*ATP6V0A4_rna - 0.1449*ATP6V0D2_rna + 0.002*ATP6V0E1_cnv - 0.0312*B4GALT7_cnv + 0.0237*BAI1_rna + 0.0231*BAP1_mut + 0.1301*BNC1_rna + 0.0571*BNIP1_cnv + 0.0605*BOD1_cnv - 0.0765*BSND_rna + 0.0571*BTNL3_cnv + 0.069*BTNL8_cnv + 0.0593*BTNL9_cnv + 0.0175*C11orf86_rna + 0.1499*C15orf59_rna + 0.0317*C17orf55_rna + 0.1591*C1QTNF3_rna + 0.1171*C4BPA_rna - 0.8872*C4orf7_rna - 0.0397*C8orf22_rna - 0.0622*CA1_rna - 0.1865*CALCR_rna + 0.0787*CANX_cnv - 0.2479*CAPN6_rna - 0.0093*CBY3_cnv + 0.0931*CCBE1_rna - 0.2213*CDH17_rna + 0.1271*CDH22_rna + 0.1433*CDHR2_cnv - 0.2913*CELF5_rna + 0.2208*CHRDL1_rna - 0.0214*CIDEC_rna + 0.2182*CILP_r

\mathtt{\text{-0.0224*AACSP1\_cnv + 0.0164*ACHE\_rna - 0.1246*ACTG2\_rna - 0.2214*ADAM6\_rna + 0.0751*ADAMTS2\_cnv + 0.0429*ADCYAP1\_rna + 0.092*ADRB1\_rna - 0.0548*AGR2\_rna - 0.0251*AGT\_rna - 0.0959*AGXT2\_rna - 0.1538*ALPI\_rna - 0.0033*ANKRD1\_rna + 0.3331*APCDD1L\_rna - 0.2505*AREG\_rna - 0.2206*ARL10\_cnv + 0.1533*ARPP21\_rna - 0.2197*ART5\_rna - 0.0648*ASAM\_rna + 0.024*ATP13A4\_rna - 0.1172*ATP6V0A4\_rna - 0.1449*ATP6V0D2\_rna + 0.002*ATP6V0E1\_cnv - 0.0312*B4GALT7\_cnv + 0.0237*BAI1\_rna + 0.0231*BAP1\_mut + 0.1301*BNC1\_rna + 0.0571*BNIP1\_cnv + 0.0605*BOD1\_cnv - 0.0765*BSND\_rna + 0.0571*BTNL3\_cnv + 0.069*BTNL8\_cnv + 0.0593*BTNL9\_cnv + 0.0175*C11orf86\_rna + 0.1499*C15orf59\_rna + 0.0317*C17orf55\_rna + 0.1591*C1QTNF3\_rna + 0.1171*C4BPA\_rna - 0.8872*C4orf7\_rna - 0.0397*C8orf22\_rna - 0.0622*CA1\_rna - 0.1865*CALCR\_rna + 0.0787*CANX\_cnv - 0.2479*CAPN6\_rna - 0.0093*CBY3\_cnv + 0.0931*CCBE1\_rna - 0.2213*CDH17\_rna + 0.1271*CDH22\_rna + 0.1433*CDHR2\_cnv - 0.2913*CEL

(The kirc terms were not auto-ranked so I re-implemented the equation).

$$
\begin{equation}
\begin{split}
\hat{\theta}_{KAN} = & + 0.43 \cdot \text{MT1X}_{rna} \quad (\sigma=0.42) \, \\
& +0.34 \cdot \text{DDX43}_{rna} \quad (\sigma=0.34) \\
& +0.23 \cdot \text{CWH43}_{rna} \quad (\sigma=0.31) \\
& +0.22 \cdot \text{CILP}_{RNA} \quad (\sigma=0.31) \\
& -0.24 \cdot \text{LOC153328}_{RNA} \quad (\sigma=0.29) \\
& -0.21 \cdot \text{CYP3A7}_{RNA} \quad (\sigma=0.28) \\
& + \text{less significant terms},
\end{split}
\end{equation}
$$